In [1]:
!nvidia-smi

Fri Nov  7 19:24:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.09              Driver Version: 580.82.09      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:04:00.0 Off |                  N/A |
| 46%   50C    P8             26W /  370W |       1MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
import numpy as np
import re
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import torch.quantization.quantize_fx as quantize_fx

from glob import glob
from tqdm import tqdm
from pprint import pprint
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from sklearn.metrics import f1_score 

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNAttenClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
from code_base.utils.main_utils import get_device
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!ls ../data/predictions -lt

/venv/main/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=335803) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


total 0


# Clean logdirs

In [ ]:
# def filt_criteria(input):
#     num_folds = sum("fold_" in el for el in glob(pjoin(input, "*")))
#     if num_folds > 0 and num_folds < 5:
#         print(input, num_folds)
#         return False
#     else:
#         return True
    
# all_logdirs = glob("../logdirs/*")
# logdirs_to_remove = [el for el in all_logdirs if not filt_criteria(el)]

In [ ]:
# for el in logdirs_to_remove[:-1]:
#     !rm {el} -rf

# Config

In [6]:
!ls -lt ../logdirs/

/venv/main/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=335803) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


total 0
drwxrwxr-x 8 root root 116 Oct 31 12:17 eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005


In [ ]:
# print("Possible exps:\n\n{}".format("\n".join(os.listdir("../logdirs/"))))

In [ ]:
# bird2id_source = load_json("/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/bird2int_2025_and_prev_comps.json")
# bird2id_target = load_json("/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/bird2int_2025.json")

# id2bird_source = {v:k for k,v in bird2id_source.items()}
# id2bird_target = {v:k for k,v in bird2id_target.items()}

# REARRANGE_INDICES = np.array([
#     bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target))
# ]).astype(int)

# def prune_checkpoint_rule(inp_chkp):
#     inp_chkp["head.attention.weight"] = inp_chkp["head.attention.weight"][REARRANGE_INDICES]
#     inp_chkp["head.attention.bias"] = inp_chkp["head.attention.bias"][REARRANGE_INDICES]
    
#     inp_chkp["head.fix_scale.weight"] = inp_chkp["head.fix_scale.weight"][REARRANGE_INDICES]
#     inp_chkp["head.fix_scale.bias"] = inp_chkp["head.fix_scale.bias"][REARRANGE_INDICES]

#     return inp_chkp

In [7]:
POSTFIX = ""
EXP_NAME = "eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005"
TRAIN_PERIOD = 5
print("Possible checkpoints:\n\n{}".format("\n".join(set([os.path.basename(el) for el in glob(f"../logdirs/{EXP_NAME}/*/checkpoints/*.ckpt*") if "train" not in os.path.basename(el)]))))

Possible checkpoints:

epoch=26-step=30240-valid_roc_auc=0.985.ckpt
epoch=27-step=28980-valid_roc_auc=0.983.ckpt
epoch=17-step=21168-valid_roc_auc=0.984.ckpt
epoch=38-step=45006-valid_roc_auc=0.984.ckpt
epoch=22-step=25760-valid_roc_auc=0.986.ckpt
epoch=28-step=34104-valid_roc_auc=0.984.ckpt
epoch=20-step=23604-valid_roc_auc=0.983.ckpt
epoch=41-step=47040-valid_roc_auc=0.985.ckpt
epoch=40-step=46084-valid_roc_auc=0.983.ckpt
epoch=29-step=34620-valid_roc_auc=0.984.ckpt
epoch=36-step=38295-valid_roc_auc=0.983.ckpt
epoch=18-step=22344-valid_roc_auc=0.984.ckpt
epoch=39-step=44960-valid_roc_auc=0.983.ckpt
epoch=38-step=40365-valid_roc_auc=0.983.ckpt
last.ckpt
epoch=39-step=46160-valid_roc_auc=0.984.ckpt


In [8]:
conf_path = glob(f"../logdirs/{EXP_NAME}/code/*train_configs*.py")
assert len(conf_path) == 1
conf_path = conf_path[0]
!cat {conf_path}

/venv/main/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=335803) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


from glob import glob

import torch

from code_base.augmentations.transforms import BackgroundNoise, OneOf
from code_base.callbacks import ROC_AUC_Score
from code_base.datasets import WaveAllFileDataset, WaveDataset
from code_base.forwards import MultilabelClsForwardLongShort
from code_base.models import RandomFiltering, WaveCNNAttenClasifier
from code_base.train_functions.train_lightning import lightning_training

B_S = 64
TRAIN_PERIOD = 5.0
N_EPOCHS = 50
ROOT_PATH = "/workspace/birdsongs"
LATE_NORMALIZE = True
MAXIMIZE_METRIC = True
MAIN_METRIC = "valid_roc_auc"
PATH_TO_JSON_MAPPING = "data/bird2int_xcm.json"
PRECOMPUTE = False
REPLACE_PATHES = ("xcm", "xcm_hdf5")
DEBUG = False
N_CORES = 12


CONFIG = {
    "seed": 1243,
    "df_path": "data/train_classifier_xcm.csv",
    "split_path": "data/cv_split_xcm_group_recordists.npy",
    "exp_name": "eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005",
    "files_to_save": (glob("co

In [30]:
CONFIG = {
    # Inference Class
    "use_sigmoid": False,
    # Data config
    "train_df_path": "../data/train_classifier_xcm.csv",
    "split_path": "../data/cv_split_xcm_group_recordists.npy",
    "n_folds":5,
    "train_data_root":"/workspace/birdsongs",
    "test_data_root":"/workspace/birdsongs",
    "label_map_data_path": "../data/bird2int_xcm.json",
    "scored_birds_path": None, 
    "lookback": None,
    "lookahead": None,
    "segment_len": 5,
    "step": None,
    "late_normalize": True,
    "add_dataset_config": None,
    # Model config
    "exp_name": EXP_NAME,
    "model_class": WaveCNNAttenClasifier,
    "model_config": dict(
            backbone="eca_nfnet_l0",
            mel_spec_paramms={
                "sample_rate": 32000,
                "n_mels": 128,
                "f_min": 20,
                "n_fft": 2048,
                "hop_length": 512,
                "normalized": True,
            },
            head_config={
                "p": 0.5,
                "num_class": 412,
                "train_period": TRAIN_PERIOD,
                "infer_period": TRAIN_PERIOD,
            },
            exportable=True,
            fixed_amplitude_to_db=True,
        ), 
    "chkp_name": "last.ckpt",
    "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
    "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
    "delete_prefix": "model.",
    "n_swa_models": 1,
    "model_output_key": "clipwise_pred_long",
}

if CONFIG.get("use_sed_mode", False):
    assert CONFIG["step"] is not None
else:
    assert CONFIG["step"] is None
    
if "folds" not in CONFIG:
    CONFIG["folds"] = list(range(CONFIG["n_folds"]))
    

# Data

In [31]:
bird2id = load_json(CONFIG["label_map_data_path"])

In [32]:
df = pd.read_csv(CONFIG["train_df_path"])
split = np.load(CONFIG["split_path"], allow_pickle=True)
val_df = [df.iloc[split[i][1]].reset_index(drop=True) for i in CONFIG["folds"]]

In [33]:
# val_df = [df[df["data_root_id"].isin(["add_train_audio_from_xeno_canto_11052025", "csa_train_audio_parsed_11052025"])].reset_index(drop=True) for i in CONFIG["folds"]]

In [34]:
val_ds_conig = {
    "root": CONFIG["train_data_root"],
    "label_str2int_mapping_path": CONFIG["label_map_data_path"],
    "use_audio_cache": True,
    "test_mode": True,
    "n_cores": 64,
    "verbose": False,
    "segment_len": CONFIG["segment_len"],
    "lookback":CONFIG["lookback"],
    "lookahead":CONFIG["lookahead"],
    "sample_id": None,
    "late_normalize": CONFIG["late_normalize"],
    "step": CONFIG["step"],
    "duration_col": "duration_s"
    # "validate_sr": 32_000,
}
if CONFIG.get("add_dataset_config") is not None:
    val_ds_conig.update(CONFIG["add_dataset_config"])
loader_config = {
    "batch_size": 64,
    "drop_last": False,
    "shuffle": False,
    "num_workers": 0,
}

In [35]:
ds_val = [WaveAllFileDataset(df=df, **val_ds_conig) for df in val_df]

Validating that all files exist


100%|██████████| 23490/23490 [00:00<00:00, 154069.73it/s]


Validating that all files exist


100%|██████████| 18100/18100 [00:00<00:00, 154158.52it/s]


Validating that all files exist


100%|██████████| 14485/14485 [00:00<00:00, 154916.40it/s]


Validating that all files exist


100%|██████████| 17803/17803 [00:00<00:00, 153003.14it/s]


Validating that all files exist


100%|██████████| 15915/15915 [00:00<00:00, 153589.26it/s]


In [36]:
loader_val = [torch.utils.data.DataLoader(
    ds,
    **loader_config,
)for ds in ds_val]

In [37]:
len(loader_val)

5

In [38]:
val_ds_conig

{'root': '/workspace/birdsongs',
 'label_str2int_mapping_path': '../data/bird2int_xcm.json',
 'use_audio_cache': True,
 'test_mode': True,
 'n_cores': 64,
 'verbose': False,
 'segment_len': 5,
 'lookback': None,
 'lookahead': None,
 'sample_id': None,
 'late_normalize': True,
 'step': None,
 'duration_col': 'duration_s'}

# Model

In [39]:
def create_model_and_upload_chkp(
    model_class,
    model_config,
    model_device,
    model_chkp_root,
    model_chkp_basename=None,
    model_chkp_regex=None,
    delete_prefix=None,
    swa_sort_rule=None,
    n_swa_to_take=3,
    prune_checkpoint_func=None
):
    if model_chkp_basename is None:
        basenames = os.listdir(model_chkp_root)
        checkpoints = []
        for el in basenames:
            matches = re.findall(model_chkp_regex, el)
            if not matches:
                continue
            parsed_dict = {key: value for key, value in matches}
            parsed_dict["name"] = el
            checkpoints.append(parsed_dict)
        print("SWA checkpoints")
        pprint(checkpoints)
        checkpoints = sorted(checkpoints, key=swa_sort_rule)
        checkpoints = checkpoints[:n_swa_to_take]
        print("SWA sorted checkpoints")
        pprint(checkpoints)
        if len(checkpoints) > 1:
            checkpoints = [
                torch.load(os.path.join(model_chkp_root, el["name"]), map_location="cpu")["state_dict"] for el in checkpoints
            ]
            t_chkp = avarage_weights(
                nn_weights=checkpoints,
                delete_prefix=delete_prefix
            )
        else:
            chkp_path = os.path.join(model_chkp_root, checkpoints[0]["name"])
            print("vanilla model")
            print("Loading", chkp_path)
            t_chkp = torch.load(
                chkp_path, 
                map_location="cpu"
            )["state_dict"]
            if delete_prefix is not None:
                t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)
    else:
        chkp_path = os.path.join(model_chkp_root, model_chkp_basename)
        print("vanilla model")
        print("Loading", chkp_path)
        t_chkp = torch.load(
            chkp_path, 
            map_location="cpu"
        )["state_dict"]
        if delete_prefix is not None:
            t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)

    if prune_checkpoint_func is not None:
        t_chkp = prune_checkpoint_func(t_chkp)
    t_model = model_class(**model_config, device=model_device) 
    print("Missing keys: ", set(t_model.state_dict().keys()) - set(t_chkp))
    print("Extra keys: ",  set(t_chkp) - set(t_model.state_dict().keys()))
    t_model.load_state_dict(t_chkp, strict=False)
    t_model.eval()
    return t_model

In [40]:

model = [create_model_and_upload_chkp(
    model_class=CONFIG["model_class"],
    model_config=CONFIG['model_config'],
    model_device=get_device(),
    model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/fold_{m_i}/checkpoints",
    # model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/checkpoints",
    model_chkp_basename=CONFIG["chkp_name"] if CONFIG["swa_checkpoint_regex"] is None else None,
    model_chkp_regex=CONFIG.get("swa_checkpoint_regex"),
    swa_sort_rule=CONFIG.get("swa_sort_rule"),
    n_swa_to_take=CONFIG.get("n_swa_models", 3),
    delete_prefix=CONFIG.get("delete_prefix"),
    prune_checkpoint_func=CONFIG.get("prune_checkpoint_func")
) for m_i in range(CONFIG["n_folds"])]


SWA checkpoints
[{'name': 'epoch=27-step=28980-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'},
 {'name': 'epoch=36-step=38295-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'},
 {'name': 'epoch=38-step=40365-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'}]
SWA sorted checkpoints
[{'name': 'epoch=27-step=28980-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'}]
vanilla model
Loading ../logdirs/eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005/fold_0/checkpoints/epoch=27-step=28980-valid_roc_auc=0.983.ckpt
STFT kernels created, time used = 0.7526 seconds
Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
SWA checkpoints
[{'name': 'epoch=22-step=25760-valid_roc_auc=0.986.ckpt',
  'valid_roc_auc': '0.986'},
 {'name': 'epoch=26-step=30240-valid_roc_auc=0.985.ckpt',
  'valid_roc_auc': '0.985'},
 {'name': 'epoch=41-step=47040-valid_roc_auc=0.985.ckpt',
  'valid_roc_auc': '0.985'}]
SWA sorted 

# Inference Class

In [41]:
inference_class = BirdsInference(
    device="cuda",
    verbose_tqdm=True,
    use_sigmoid=CONFIG["use_sigmoid"],
    model_output_key=CONFIG["model_output_key"],
)

In [42]:
bird2id = load_json(CONFIG["label_map_data_path"])

# Val Predict

In [ ]:
all_predicted_folds_df = []
for fold_model, fold_loader in zip(model, loader_val):
    fold_filenames = fold_loader.dataset.df[fold_loader.dataset.name_col].copy()
    assert len(set(fold_filenames)) == len(fold_filenames)
    fold_preds, fold_dfidx, fold_end = inference_class.predict_test_loader(
        nn_models=[fold_model],
        data_loader=fold_loader
    )
    all_predicted_folds_df.append(compose_submission_dataframe(
        probs=fold_preds,
        dfidxs=fold_dfidx,
        end_seconds=fold_end,
        filenames=fold_filenames,
        bird2id=bird2id
    ))

  0%|          | 0/5033 [00:00<?, ?it/s]

 15%|█▌        | 759/5033 [34:43<2:18:58,  1.95s/it] Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7697eaec19d0>>
Traceback (most recent call last):
  File "/workspace/bird_sound_classification_2025/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
 15%|█▌        | 770/5033 [35:07<2:06:43,  1.78s/it]

In [ ]:
for i in range(len(all_predicted_folds_df)):
    all_predicted_folds_df[i]["fold"] = i

In [ ]:
all_predicted_folds_df = pd.concat(all_predicted_folds_df).reset_index(drop=True)

In [ ]:
all_predicted_folds_df

In [ ]:
plt.title("Most 'Probable' class probability distribution")
plt.hist(all_predicted_folds_df.iloc[:,1:-1].values.max(axis=1), bins=30)
plt.show()

print(
    "Max Prob: ", all_predicted_folds_df.iloc[:,1:-1].values.max(), 
    "Min Prob: ", all_predicted_folds_df.iloc[:,1:-1].values.min(),
    "Median Prob: ", np.median(all_predicted_folds_df.iloc[:,1:-1].values)
)

# Save Results

In [ ]:
!ls ../data/predictions -lt

In [ ]:
SAVE_PATH = os.path.join(
    "../data/predictions",
    EXP_NAME + POSTFIX + ".csv"
)
print(SAVE_PATH)
all_predicted_folds_df.to_csv(SAVE_PATH, index=False)

In [ ]:
# !rm ../data/bird_clef_2025_predictions/__bird_clef_2025_add_data_v3__tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_AdamW1e4_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromPrebs1_PseudoF2PT05MT01P04I2_AddRareBirdsNoLeak.csv

# Merge Results

In [ ]:
df_1 = pd.read_csv(
    "../data/bird_clef_2025_predictions/eca_nfnet_l0_Exp_noamp_64bs_5sec_BasicAug_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromPreca4_PseudoF2PT05MT01P04I2_MinorOverSampleV1.csv"
)
df_2 = pd.read_csv(
    "../data/bird_clef_2025_predictions/tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_BasicAug_EqualBalancing_AdamW1e4_CosBatchLR1e6_Epoch50_FocalBCELoss_LSF1005_FromPrebs1_PseudoF2PT05MT01P04I2.csv"
)

In [ ]:
assert (df_2["row_id"] == df_1["row_id"]).all()

In [ ]:
df_merged = df_1.copy()

In [ ]:
df_merged.iloc[:,1:-1] = (
    df_1.iloc[:,1:-1] + df_2.iloc[:,1:-1]
) / 2

In [ ]:
df_merged.to_csv(
    "../data/bird_clef_2025_predictions/PseudI2_eca_l0_tf_efficientnetv2_s.csv",
    index=False
)

In [ ]:
row_id_vc = df_merged["row_id"].value_counts()

In [ ]:
df_merged = df_merged[~df_merged["row_id"].isin(row_id_vc[row_id_vc > 1].index)].reset_index(drop=True)

In [ ]:
df_merged["row_id"].value_counts()

In [ ]:
df_merged.to_csv(
    "../data/bird_clef_2025_predictions/PseudI2_eca_l0_tf_efficientnetv2_s__nodupl.csv",
    index=False
)